# Practica 2B (Ascensores)
##### Alberto García Doménech - Pablo Daurell Marina (Grupo 03)

In [184]:
# Restriccion: Una persona por ascensor
# El último ascensor siempre es rápido (se  mueve por plantas pares)
# estado_ini = ( (personas), (ascensores) )
estado_ini = (
    (2,4,2,8,1), (2,0,1)
)

# estado_goal = (personas)
estado_goal = (3,11,12,1,9)

# Acciones
# Llamar ascensor persona (mueve un ascensor vacio a una planta)
# Subir ascensor persona (sube una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)
# Bajar ascensor persona (baja una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)

In [149]:
from search import *

class Ascensores(Problem):
    
    def __init__(self, initial, goal):
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        personas = list(estado[0])
        ascensores = list(estado[1])
        accs = list()
        for i, persona in enumerate(personas):
            for j, ascensor in enumerate(ascensores):
                if j == len(ascensores)-1: # ASCENSOR RAPIDO
                    if persona != ascensor and persona%2 == 0:
                        accs.append(("LlamarRapido", i, j)) # Persona i (en planta par) llama a ascensor rapido
                    if persona == ascensor and int(persona) >0:
                        accs.append(("BajarRapido" , i, j)) # Bajar persona i en asccensor rapido (j)
                    if persona == ascensor and int(persona) < 12:
                        accs.append(("SubirRapido", i, j)) # Subir persona i en ascensor rapido (j)
                else: # ASCENSOR NORMAL
                    if persona != ascensor:
                        accs.append(("Llamar", i, j))            # Persona i llama a asccensor j
                    if persona == ascensor and int(persona) > 0:
                        accs.append(("Bajar", i, j))             # Bajar persona i en ascensor j
                    if persona == ascensor and int(persona) < 12:
                        accs.append(("Subir", i, j))             # Subir persona i en ascensor j
            
        return accs
    
    def result(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar" or accion[0] == "LlamarRapido":
            ascensores[a] = int(personas[p])
        elif accion[0] == "Bajar":
            ascensores[a] -= 1
            personas[p] -= 1
        elif accion[0] == "Subir":
            ascensores[a] += 1
            personas[p] += 1
        elif accion[0] == "BajarRapido":
            ascensores[a] -= 2
            personas[p] -= 2
        elif accion[0] == "SubirRapido":
            ascensores[a] += 2
            personas[p] += 2
        return (tuple(personas), tuple(ascensores))
    
    def goal_test(self, estado):
        return estado[0] == self.goal
    
    def coste_de_aplicar_accion(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar":
            coste = abs(personas[p] - ascensores[a])
        elif accion[0] == "Bajar":
            coste = 1
        elif accion[0] == "Subir":
            coste = 1
        elif accion[0] == "LlamarRapido":
            coste = abs(personas[p] - ascensores[a])/2
        elif accion[0] == "BajarRapido":
            coste = 1/2
        elif accion[0] == "SubirRapido":
            coste = 1/2
        return coste


In [150]:
p = Ascensores(estado_ini, estado_goal)
p.initial

((2, 4, 2, 8, 1), (1, 0))

In [151]:
p.goal

(3, 11, 12, 1, 9)

In [185]:
p.actions(p.initial)

[('Llamar', 0, 0),
 ('Llamar', 0, 1),
 ('LlamarRapido', 0, 2),
 ('Llamar', 1, 0),
 ('Llamar', 1, 1),
 ('LlamarRapido', 1, 2),
 ('Llamar', 2, 0),
 ('Llamar', 2, 1),
 ('Llamar', 3, 0),
 ('Llamar', 3, 1),
 ('LlamarRapido', 3, 2),
 ('Llamar', 4, 0),
 ('Llamar', 4, 1)]

In [153]:
p.result(p.initial, ("Llamar", 0, 0))

((2, 4, 2, 8, 1), (2, 0))

In [154]:
p.coste_de_aplicar_accion(p.initial, ("LlamarRapido", 1, 0))

1.5

### Heurísticas

In [156]:
# Numero de personas en una planta incorrecta a la de destino 
def wrong_pos(node):
    res = 0
    goal = estado_goal
    for i in range(len(node.state[0])):
        if goal[i] != node.state[0][i]:
            res += 1
    return res

# Distancia de cada persona a su planta destino
def p_distance(node):
    state = node.state[0]
    dist = 0
    goal = estado_goal
    for i in range(len(state)):
        if goal[i] != state[i]:
            dist += abs(state[i] - goal[i])
    return dist

# Distancia de cada persona a su planta destino (con ascensor rapido)
def p_distance2(node):
    state = node.state[0]
    dist = 0
    goal = estado_goal
    for i in range(len(state)):
        if goal[i] != state[i]:
            if state[i]%2 == 0: # Planta par
                if goal[i]%2 == 0: # Destino par
                    dist += abs(state[i] - goal[i])/2
                else: # Destino impar
                    dist += (abs(state[i] - goal[i])/2)+1
            else: # Planta impar
                if(goal[i]>state[i]): # Si tienes que subir
                    if goal[i]%2 == 0: # Destino par
                        dist += (abs((state[i]+1) - goal[i])/2)+1
                    else: # Destino impar
                        dist += (abs((state[i]+1) - (goal[i]-1))/2) + 2
                else: # Si tienes que bajar
                    if goal[i]%2 == 0: # Destino par
                        dist += (abs((state[i]-1)-goal[i])/2)+1
                    else: # Destino impar
                        dist += (abs((state[i]-1) - (goal[i]+1))/2) + 2
    return dist

### Búsqueda heurística

In [230]:
from search import best_first_graph_search

In [303]:
estado_ini = ((0,0),(0,0))
estado_goal = (12,3)
p = Ascensores(estado_ini, estado_goal)

In [336]:
%%timeit
astar_search(p, p_distance2).solution()

2.12 ms ± 19 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [337]:
%%timeit
astar_search(p, p_distance).solution()

749 µs ± 4.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [338]:
%%timeit
best_first_graph_search(p, p_distance2).solution

886 µs ± 85.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [339]:
%%timeit
best_first_graph_search(p, p_distance).solution

817 µs ± 101 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [261]:
estado_ini = ((2,4,1,8,1),(0,0))
estado_goal = (0,0,0,0,0)
p = Ascensores(estado_ini, estado_goal)

In [286]:
%%time
astar_search(p, p_distance2).solution()

Wall time: 54.8 s


[('LlamarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 0, 1),
 ('LlamarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 1, 1),
 ('Llamar', 2, 0),
 ('Bajar', 2, 0),
 ('LlamarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('Llamar', 4, 0),
 ('Bajar', 4, 0)]

In [285]:
%%time
astar_search(p, p_distance).solution()

Wall time: 10 ms


[('LlamarRapido', 0, 1),
 ('BajarRapido', 0, 1),
 ('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('LlamarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('Llamar', 2, 0),
 ('Bajar', 2, 0),
 ('Llamar', 4, 0),
 ('Bajar', 4, 0)]

In [239]:
estado_ini = ((2,4,1,8,1),(0,4,4,8,0))
estado_goal = (3,11,12,1,9)
p = Ascensores(estado_ini, estado_goal)

In [334]:
%%timeit
best_first_graph_search(p, p_distance2).solution();

853 µs ± 43.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [335]:
%%timeit
best_first_graph_search(p, p_distance).solution();

732 µs ± 64.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
